## News Sentiment Analysis and Crypto price prediction

In [ ]:
# Book 11 - Bitcoin - XG Boost Regressor Model

In [ ]:
#Importing Libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from xgboost import XGBRegressor
%matplotlib inline
from sklearn import metrics

In [9]:
# Load the news sentiment data and closing prices for Bitcoin
xgbr_df = pd.read_csv('sentiment_closing_bit.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)
xgbr_df.tail()

,title_sent,text_sent,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,,
2022-03-06,0.0,-1.0,39404.199219,39640.175781,38211.648438,38419.984375,38419.984375,1.974523e+10
2022-03-07,-1.0,-1.0,38429.304688,39430.226562,37260.203125,38062.039062,38062.039062,2.854614e+10
2022-03-08,-1.0,-1.0,38059.902344,39304.441406,37957.386719,38737.269531,38737.269531,2.577658e+10
2022-03-09,-1.0,-1.0,38742.816406,42465.671875,38706.093750,41982.925781,41982.925781,3.228412e+10
2022-03-10,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Choosing only the Relevant Columns and dropping the others
xgbr_df = xgbr_df.drop(columns=["Open", "High", "Low", "Adj Close", "text_sent"])

xgbr_df.head()

,title_sent,Close,Volume
Date,,,
2022-02-10,-1.0,43565.113281,3.214205e+10
2022-02-11,-1.0,42407.937500,2.695493e+10
2022-02-12,0.0,42244.468750,1.815239e+10
2022-02-13,-1.0,42197.515625,1.474159e+10
2022-02-14,-1.0,42586.917969,2.082778e+10


In [11]:
# Repositioning the columns
xgbr_df = xgbr_df[["Close", "title_sent", "Volume"]]
xgbr_df.head()

,Close,title_sent,Volume
Date,,,
2022-02-10,43565.113281,-1.0,3.214205e+10
2022-02-11,42407.937500,-1.0,2.695493e+10
2022-02-12,42244.468750,0.0,1.815239e+10
2022-02-13,42197.515625,-1.0,1.474159e+10
2022-02-14,42586.917969,-1.0,2.082778e+10


In [12]:
# pct change based on close value
xgbr_df["Pct_change"] = xgbr_df["Close"].pct_change()
xgbr_df.head()

,Close,title_sent,Volume,Pct_change
Date,,,,
2022-02-10,43565.113281,-1.0,3.214205e+10,NaN
2022-02-11,42407.937500,-1.0,2.695493e+10,-0.026562
2022-02-12,42244.468750,0.0,1.815239e+10,-0.003855
2022-02-13,42197.515625,-1.0,1.474159e+10,-0.001111
2022-02-14,42586.917969,-1.0,2.082778e+10,0.009228


In [13]:
# Drop null values
xgbr_df.dropna(inplace = True)
xgbr_df.tail()

,Close,title_sent,Volume,Pct_change
Date,,,,
2022-03-05,39400.585938,0.0,1.697592e+10,0.006719
2022-03-06,38419.984375,0.0,1.974523e+10,-0.024888
2022-03-07,38062.039062,-1.0,2.854614e+10,-0.009317
2022-03-08,38737.269531,-1.0,2.577658e+10,0.017740
2022-03-09,41982.925781,-1.0,3.228412e+10,0.083786


### Creating the Features `X` and Target `y` Data

In [14]:
# This function "window_data" accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number1, feature_col_number2, feature_col_number3, target_col_number):
    # Create empty lists "X_close", "title_sent", "Volume" and y
    X_close = []
    X_title_sent = []
    X_volume = []
    y = []
    for i in range(len(df) - window):
        
        # Get close, title_sent, Volume, and target in the loop
        Close = df.iloc[i:(i + window), feature_col_number1]
        title_sent = df.iloc[i:(i + window), feature_col_number2]
        Volume = df.iloc[i:(i + window), feature_col_number3]
        target = df.iloc[(i + window), target_col_number]
        
        # Append values in the lists
        X_close.append(Close)
        X_title_sent.append(title_sent)
        X_volume.append(Volume)
        y.append(target)
        
    return np.hstack((X_close,X_title_sent,X_volume)), np.array(y).reshape(-1, 1)

In [15]:
# Predict Closing Prices using a 3 day window of previous closing prices
window_size = 3

# Column index 0 is the `Close` column
# Column index 1 is the `title_sent` column
# Column index 2 is the `Volume` column
feature_col_number1 = 0
feature_col_number2 = 1
feature_col_number3 = 2
target_col_number = 0
X, y = window_data(xgbr_df, window_size, feature_col_number1, feature_col_number2, feature_col_number3, target_col_number)

In [16]:
# Use 70% of the data for training and 30% for testing
X_split = int(0.7 * len(X))
y_split = int(0.7 * len(y))

# Set X_train, X_test, y_train, t_test
X_train = X[: X_split]
X_test = X[X_split:]
y_train = y[: y_split]
y_test = y[y_split:]

# Scaling Data with `MinMaxScaler`

We will use the `MinMaxScaler` from `sklearn` to scale all values between `0` and `1`.
Note that we scale both features and target sets.

In [17]:
from sklearn.preprocessing import MinMaxScaler

In [18]:
# Use the MinMaxScaler to scale data between 0 and 1.
x_train_scaler = MinMaxScaler()
x_test_scaler = MinMaxScaler()
y_train_scaler = MinMaxScaler()
y_test_scaler = MinMaxScaler()

# Fit the scaler for the Training Data
x_train_scaler.fit(X_train)
y_train_scaler.fit(y_train)

# Scale the training data
X_train = x_train_scaler.transform(X_train)
y_train = y_train_scaler.transform(y_train)

# Fit the scaler for the Testing Data
x_test_scaler.fit(X_test)
y_test_scaler.fit(y_test)

# Scale the y_test data
X_test = x_test_scaler.transform(X_test)
y_test = y_test_scaler.transform(y_test)

In [19]:
# Create the XG Boost regressor instance
model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)

In [20]:
# Fit the model
model.fit(X_train, y_train.ravel())

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

---

---

## Model Performance

In this section, we will evaluate the model using the test data. 

We will:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart


In [21]:
# Make some predictions
predicted = model.predict(X_test)

In [22]:
# Evaluating the model
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predicted)))
print('R-squared :', metrics.r2_score(y_test, predicted))

Root Mean Squared Error: 0.2594662801266133
R-squared : 0.45132394090848027


In [23]:
# Recover the original prices instead of the scaled version
predicted_prices = y_test_scaler.inverse_transform(predicted.reshape(-1, 1))
real_prices = y_test_scaler.inverse_transform(y_test.reshape(-1, 1))

In [24]:
# Create a DataFrame of Real and Predicted values
crypto = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = xgbr_df.index[-len(real_prices): ]) 
xgbr_df.head()

,Close,title_sent,Volume,Pct_change
Date,,,,
2022-02-11,42407.937500,-1.0,2.695493e+10,-0.026562
2022-02-12,42244.468750,0.0,1.815239e+10,-0.003855
2022-02-13,42197.515625,-1.0,1.474159e+10,-0.001111
2022-02-14,42586.917969,-1.0,2.082778e+10,0.009228
2022-02-15,44575.203125,-1.0,2.272166e+10,0.046688


In [25]:
# Plot the real vs predicted values as a line chart
crypto.hvplot(title = "Real vs Predicted values of Bitcoin")

:NdOverlay   [Variable]
   :Curve   [Date]   (value)